# Purpose

This notebook implements basic data cleaning.

* fill None with "None" or -255 if numeric
* to convert long string categorical data into integer
* remove list of items which do not appear in train and test data

After data cleaning, data size gets smaller and become easier for analysis.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

In [2]:
test_df = pd.read_parquet("data/test.parquet")

In [3]:
train_df = pd.read_parquet("data/train.parquet")

In [4]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [5]:
n_train = train_df.shape[0]
n_test = test_df.shape[0]

In [6]:
member_df = pd.read_parquet("data/members.parquet")

In [7]:
n_member = member_df.shape[0]
n_member

34403

In [8]:
member_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,None,7,20110820,20170920
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,None,7,20150628,20170622
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,None,4,20160411,20170712
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,None,9,20150906,20150907
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,None,4,20170126,20170613


In [9]:
song_df = pd.read_parquet("data/songs.parquet")

In [10]:
song_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,None,None,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [11]:
n_song = song_df.shape[0]
n_song

2296320

## Fill None 

In [12]:
for c in song_df.columns:
    filter1 = song_df[c].isna()
    if song_df.dtypes[c] == object:
        song_df.loc[filter1, c] = "None"
        
    else:
        song_df.loc[filter1, c] = -255
    

In [13]:
for c in train_df.columns:
    filter1 = train_df[c].isna()
    if train_df.dtypes[c] == object:
        train_df.loc[filter1, c] = "None"


In [14]:
for c in test_df.columns:
    filter1 = test_df[c].isna()
    if test_df.dtypes[c] == object:
        test_df.loc[filter1, c] = "None"

## Remove songs and members which does not appear in train and test data

In [15]:
song_set= set(pd.concat([train_df["song_id"], test_df["song_id"]]).unique())

In [16]:
filter1 = song_df["song_id"].apply(lambda x: x in song_set)
song_df = song_df.loc[filter1]

In [17]:
msno_set = set(pd.concat([train_df["msno"], test_df["msno"]]).unique())

In [18]:
filter1 = member_df["msno"].apply(lambda x: x in msno_set)
member_df = member_df.loc[filter1]

In [19]:
n_song = song_df.shape[0]
n_member = member_df.shape[0]

## Replace string (song id, and member id) with integer

In [20]:
def string_convert(val, dic):
    try:
        val = dic[val]
    except KeyError:
        val = -1
        
    return val

### Member data

In [21]:
member_df["userID"] = np.arange(member_df.shape[0])

In [22]:
msno_array = member_df["msno"].to_numpy()

In [23]:
member_idx_dic = {}
for i in range(n_member):
    member_idx_dic[msno_array[i]] = i

In [24]:
train_df["userID"] = train_df["msno"].apply(lambda x: string_convert(x,member_idx_dic ))

In [25]:
test_df["userID"] = test_df["msno"].apply(lambda x: string_convert(x,member_idx_dic ))

### Song Data

In [26]:
song_df["artist_id"] = song_df["artist_name"].astype("category").cat.codes

In [27]:
song_df["itemID"] = np.arange(song_df.shape[0])

In [28]:
song_idx_dic = {}
song_id_array = song_df["song_id"].to_numpy()
for i in range(n_song):
    song_idx_dic[song_id_array[i]] = i

In [29]:
train_df["itemID"] = train_df["song_id"].apply(lambda x: string_convert(x,song_idx_dic ))

In [30]:
test_df["itemID"] = test_df["song_id"].apply(lambda x: string_convert(x,song_idx_dic ))

In [31]:
train_df.rename({"target": "rating"}, axis =  1, inplace = True)

### Check

In [32]:
train_df.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,rating,userID,itemID
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,21893,335731
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,26052,357849
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,26052,210391
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1,26052,308876
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1,21893,222980


In [33]:
test_df.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type,userID,itemID
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,29464,187683
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,29464,2101
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,None,song-based-playlist,24266,10113
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio,22501,78829
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio,22501,42570


In [34]:
member_df.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,userID
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,None,7,20110820,20170920,0
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,None,7,20150628,20170622,1
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,None,4,20160411,20170712,2
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,None,9,20150906,20150907,3
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,None,4,20170126,20170613,4


In [35]:
song_df.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,artist_id,itemID
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0,41671,0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0,3365,1
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,None,None,31.0,29563,2
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0,29342,3
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0,44699,4


## drop string and save file

In [36]:
train_df.drop(["msno", "song_id"], axis = 1).to_parquet("data_p1/train_p1.parquet", index = False)

In [37]:
test_df.drop(["msno", "song_id"], axis = 1).to_parquet("data_p1/test_p1.parquet", index = False)

In [38]:
member_df.drop(["msno"], axis = 1).to_parquet("data_p1/members_p1.parquet", index = False)

In [39]:
song_df.drop(["song_id"], axis = 1).to_parquet("data_p1/songs_p1.parquet", index = False)